In [1]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [2]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [3]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [4]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# occupation = 'athletesAndSportsCompetitors'
# occupation = 'audiovisualEquipmentInstallerAndRepairers'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [5]:
# set alpha as AI quality metric
n = 100
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

### Initialize input-output paths

In [6]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_M.csv'

# Naive DAG
input_path = f'{occupation_folder}/{occupation}_N_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_N.csv'

# # Conditioned Naive DAG
# input_path = f'{occupation_folder}/{occupation}_CN_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CN.csv'

# # First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_FLT.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CFLT.csv'

# # Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_P.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CP.csv'

In [7]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target,comment
0,Converse with customer to determine destinatio...,Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
1,"Compute cost of travel and accommodations, usi...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
2,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...",The worker booking transportation and hotel re...
3,"Plan, describe, arrange, and sell itinerary to...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
4,Collect payment for transportation and accommo...,Print or request transportation carrier ticket...,The worker printing or requesting transportati...
5,Converse with customer to determine destinatio...,"Compute cost of travel and accommodations, usi...",The worker computing the cost of travel and ac...
6,Converse with customer to determine destinatio...,"Book transportation and hotel reservations, us...",The worker responsible for booking transportat...
7,Converse with customer to determine destinatio...,"Plan, describe, arrange, and sell itinerary to...",The worker planning and arranging travel packa...
8,Converse with customer to determine destinatio...,Provide customer with brochures and publicatio...,The worker providing brochures and travel info...
9,Converse with customer to determine destinatio...,Print or request transportation carrier ticket...,The worker responsible for printing or request...


In [8]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Collect payment for transportation and accommodations from customer.',
 1: 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 2: 'Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tours costs.',
 3: 'Book transportation and hotel reservations, using computer or telephone.',
 4: 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 5: 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 6: 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 7: 'Record and maintain information on clients, vendors, and travel packages.',
 8: '"Target"'}

In [9]:
def find_neighbors(adjacency_matrix):
    # Get the number of nodes (n) from the shape of the adjacency matrix
    n = adjacency_matrix.shape[0]
    
    # Initialize an empty dictionary to store the neighbors for each node
    neighbors = {i: [] for i in range(n)}
    
    # Loop through each entry in the adjacency matrix
    for i in range(n):
        for j in range(n):
            # If there's an edge from i to j or from j to i, add j to the neighbors of i
            if adjacency_matrix[i, j] == 1 or adjacency_matrix[j, i] == 1:
                if j not in neighbors[i]:  # Avoid duplicate neighbors
                    neighbors[i].append(j)
                if i not in neighbors[j]:  # Ensure symmetry in the undirected version
                    neighbors[j].append(i)
    
    return neighbors

In [10]:
# def create_inactive_node_neighbor_subset_combinations(inactive_neighbors_valid_subsets_dict):
#     # Step 1: Extract lists from dictionary and remove duplicates within lists
#     all_lists = [list(set(item)) for sublist in inactive_neighbors_valid_subsets_dict.values() for item in sublist]
#     print(f'Number of lists extracted: {len(all_lists)}')

#     # Step 2: Create all combinations of lists across keys
#     output_set = set()
#     for r in range(1, len(all_lists) + 1):
#         combinations = itertools.combinations(all_lists, r)
#         for combo in combinations:
#             # Flatten the combination of lists
#             flattened_combo = list(itertools.chain(*combo))
#             # Remove duplicates within the flattened list and sort for consistency
#             unique_combo = tuple(sorted(set(flattened_combo)))
#             # Add the unique combination to the output set
#             output_set.add(unique_combo)

#     # Convert the set back to a list of lists
#     output_list = [list(combo) for combo in output_set]
#     return sorted(output_list, key=len)



def create_inactive_node_neighbor_subset_combinations(inactive_neighbors_valid_subsets_dict):
    # Step 1: Extract unique lists from the dictionary values
    all_lists = [list(set(item)) for sublist in inactive_neighbors_valid_subsets_dict.values() for item in sublist]
    print(f'Number of lists extracted: {len(all_lists)}')

    # Step 2: Create all combinations and directly add unique elements
    output_set = set()
    
    # Instead of recomputing length and duplicates, work with unique sets directly
    all_combinations = []
    
    for r in range(1, len(all_lists) + 1):
        for combo in itertools.combinations(all_lists, r):
            # Convert each combination to a flattened tuple of sorted unique elements
            flattened_combo = tuple(sorted(set(itertools.chain(*combo))))
            output_set.add(flattened_combo)  # Add to set to ensure uniqueness
    
    # Convert the set back to sorted list of lists and return the result
    output_list = [list(combo) for combo in output_set]
    
    return sorted(output_list, key=len)

In [11]:
# def get_valid_execution_plans(adjacency_matrix):

#     def valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict, memory_dict, partition, current_plan, valid_execution_plans=[]):
#         print(f'\npartition {partition}')
#         print(f'current_plan {current_plan}')

#         # if partition already in memory return its value
#         try:
#             if len(memory_dict[tuple(sorted(partition))]) > 0:
#                 print(f'partition {partition} already calculated:', memory_dict[tuple(sorted(partition))])
#                 return memory_dict[tuple(sorted(partition))]
            
#         # if partition not in memory, get valid subsets of partition
#         except KeyError:
#             # get inactive neighbors of node
#             neighbors_list = list(dict.fromkeys([value for key in partition if key in neighbors_dict for value in neighbors_dict[key]]))
#             print(f'neighbors of partition {partition}:', neighbors_list)

#             inactive_neighbors_list = [neighbor for neighbor in neighbors_list if active_dict[neighbor] == False]
#             print(f'inactive neighbors of partition {partition}:', inactive_neighbors_list)

#             # if partition has no outgoing edges return partition and empty list
#             if len(inactive_neighbors_list) == 0:
#                 # if all nodes covered then current plan is a valid execution plan
#                 covered_nodes = [num for sublist in current_plan for num in sublist]
#                 if len(covered_nodes) == n:
#                     print(f'partition {partition} is has no outgoing edges, add current plan as a valid execution plan:', current_plan)
#                     valid_execution_plans.append(current_plan)
#                 else:
#                     inactive_nodes = [node for node, status in active_dict.items() if not status]
#                     print(f'---->inactive_nodes: {inactive_nodes}<----')
#                     print(f'partition {partition} is has no outgoing edges, but not all nodes covered:', current_plan)

#                     active_dict_copy = active_dict.copy()
#                     memory_dict_copy = memory_dict.copy()

#                     active_dict_copy[min(inactive_nodes)] = True

#                     # start from next inactive node in the graph
#                     next_node = min(inactive_nodes)
#                     next_plan = current_plan + [[next_node]]
#                     print(f'\nnext_plan:', next_plan)

#                     valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict_copy, memory_dict_copy, [next_node], next_plan, valid_execution_plans)

#             else:
#                 # create all subsets of inactive neighbors to loop over
#                 inactive_neighbor_subsets = []
#                 for r in range(len(inactive_neighbors_list) + 1):
#                     inactive_neighbor_subsets.extend(itertools.combinations(inactive_neighbors_list, r))
#                 inactive_neighbor_subsets = [list(subset) for subset in inactive_neighbor_subsets if len(subset) > 0]
#                 print(f'inactive neighbor subsets:', inactive_neighbor_subsets)


#                 for neighbor_partition in inactive_neighbor_subsets:
#                     print(f'\n-----------Running partition {partition}, neighbor partition {neighbor_partition}-----------')

#                     # make a copy of active dict and memory dict to make changes only to nodes in the current neighbor partition
#                     active_dict_copy = active_dict.copy()
#                     memory_dict_copy = memory_dict.copy()

#                     # set neighbor to active and get valid subsets of neighbor
#                     for neighbor in neighbor_partition:
#                         active_dict_copy[neighbor] = True
#                     print(f'\n\n>>>>>>>>>active dict: {active_dict_copy}<<<<<<<<<')

#                     # if neighbor partition is singleton consider two extentions [partition, neighbor_partition] and [partition], [neighbor_partition], else consider only [partition, neighbor_partition]
#                     next_plan = current_plan[:-1] + [current_plan[-1] + neighbor_partition]
#                     print(f'next_plan:', next_plan)
#                     valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict_copy, memory_dict_copy, neighbor_partition, next_plan, valid_execution_plans)
                    
#                     if len(neighbor_partition) == 1:
#                         next_plan = current_plan + [neighbor_partition]
#                         print(f'\nnext_plan:', next_plan)
#                         valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict_copy, memory_dict_copy, neighbor_partition, next_plan, valid_execution_plans)

#                 # return valid_execution_plans
#                 return valid_execution_plans

    
#     # subset adjacency matrix to exclude Target node
#     non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()
    
#     # get neighbors of nodes
#     neighbors_dict = find_neighbors(non_target_adjacency_matrix)

#     # get number of non-Target nodes
#     n = non_target_adjacency_matrix.shape[0]

    
#     # create active dictionary
#     global_active_dict = {i: False for i in range(n)}

#     # set node as active
#     global_active_dict[0] = True

#     # initialize dict for valid subsets of nodes (and also partitions) to act as memory
#     global_memory_dict = {}

#     # get valid execution plans of DAG
#     valid_execution_plans = valid_execution_plans_recursive(non_target_adjacency_matrix, neighbors_dict, global_active_dict, global_memory_dict, [0], [[0]], [])

#     # sort number within subsets
#     valid_execution_plans = [[sorted(inner_list) for inner_list in outer_list] for outer_list in valid_execution_plans]

#     return valid_execution_plans

In [12]:
def get_valid_execution_plans(adjacency_matrix):

    def valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict, memory_dict, partition):
        print(f'\ncurrent partition {partition}')
        #print(f'previous partition {previous_partition}')



        active_nodes_tuple = tuple(1 if active else 0 for active in active_dict.values())
        partition_sorted_tuple = tuple(sorted(partition))



        # if partition already in memory return its value
        try:
            if len(memory_dict[(active_nodes_tuple, partition_sorted_tuple)]) > 0:
                print('*****************')
                print(f'partition {partition} already calculated:', memory_dict[(active_nodes_tuple, partition_sorted_tuple)])
                print('*****************')
                return memory_dict[(active_nodes_tuple, partition_sorted_tuple)]
            
        # if partition not in memory, get valid subsets of partition
        except KeyError:
            # get inactive neighbors of node
            neighbors_list = list(dict.fromkeys([value for key in partition if key in neighbors_dict for value in neighbors_dict[key]]))
            print(f'neighbors of partition {partition}:', neighbors_list)

            inactive_neighbors_list = [neighbor for neighbor in neighbors_list if active_dict[neighbor] == False]
            print(f'inactive neighbors of partition {partition}:', inactive_neighbors_list)

            # if partition has no outgoing edges return partition and empty list
            if len(inactive_neighbors_list) == 0:
                # if all nodes covered then current partition gives a valid execution plan
                print(f'active nodes tuple:', active_nodes_tuple)
                print(f'sum of active nodes tuple:', sum(active_nodes_tuple))
                if sum(active_nodes_tuple) == n:
                    memory_dict[(active_nodes_tuple, partition_sorted_tuple)] = [[partition]]
                    print(f'partition {partition} has no outgoing edges, update memory dict:', memory_dict)
                    return [[partition]]
                else:
                    inactive_nodes = [node for node, status in active_dict.items() if not status]
                    print(f'---->inactive_nodes: {inactive_nodes}<----')
                    print(f'partition {partition} has no outgoing edges, but not all nodes covered')

                    active_dict_copy = active_dict.copy()

                    # start from next inactive node in the graph
                    active_dict_copy[min(inactive_nodes)] = True
                    next_node = min(inactive_nodes)

                    valid_subset_matrix_extentions = valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict_copy, memory_dict, [next_node])
                    
                    # add partition to valid subset matrix extentions
                    # only add split version cause current partition has no outgoing edges
                    subset_matrix_extention_plans = []
                    for extention in valid_subset_matrix_extentions:
                        print(f'\nextention:', extention)
                        print(f'partition:', partition)
                    
                        split_extention = [partition] + extention
                        print(f'split extention:', split_extention)
                        subset_matrix_extention_plans.append(split_extention)

                    print(f'subset matrix extention plans:', subset_matrix_extention_plans)
                    return subset_matrix_extention_plans


            else:
                # create all subsets of inactive neighbors to loop over
                inactive_neighbor_subsets = []
                for r in range(len(inactive_neighbors_list) + 1):
                    inactive_neighbor_subsets.extend(itertools.combinations(inactive_neighbors_list, r))
                inactive_neighbor_subsets = [list(subset) for subset in inactive_neighbor_subsets if len(subset) > 0]
                print(f'inactive neighbor subsets:', inactive_neighbor_subsets)


                extention_plans = []
                for neighbor_partition in inactive_neighbor_subsets:
                    print(f'\n-----------Running partition {partition}, neighbor partition {neighbor_partition}-----------')

                    # make a copy of active dict and memory dict to make changes only to nodes in the current neighbor partition
                    active_dict_copy = active_dict.copy()

                    # set neighbor to active and get valid subsets of neighbor
                    for neighbor in neighbor_partition:
                        active_dict_copy[neighbor] = True
                    print(f'\n\n>>>>>>>>>active dict: {active_dict_copy}<<<<<<<<<')


                    neighbor_extention = valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, active_dict_copy, memory_dict, neighbor_partition)

                    
                    for extention in neighbor_extention:
                        print(f'\nextention:', extention)
                        print(f'partition:', partition)

                        # split extention (only for singleton neighbor partition)
                        if len(neighbor_partition) == 1:
                            split_extention = [partition] + extention
                            print(f'split extention:', split_extention)
                            extention_plans.append(split_extention)

                        # joint extention
                        joint_extention = [partition + extention[0]] + extention[1:]
                        print(f'joint extention:', joint_extention)
                        extention_plans.append(joint_extention)

                    print(f'extention plans:', extention_plans)
                    print(f'length of extention plans:', len(extention_plans))

                    # update memory dict
                    memory_dict[(active_nodes_tuple, partition_sorted_tuple)] = extention_plans
                    print(f'update memory dict:', memory_dict)

                # return extention_plans
                return extention_plans
                
    
    # subset adjacency matrix to exclude Target node
    non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()
    
    # get neighbors of nodes
    neighbors_dict = find_neighbors(non_target_adjacency_matrix)
    
    # get number of non-Target nodes
    n = non_target_adjacency_matrix.shape[0]

    
    # create active dictionary
    global_active_dict = {i: False for i in range(n)}

    # set node as active
    global_active_dict[0] = True

    # initialize dict for valid subsets of nodes (and also partitions) to act as memory
    memory_dict = {}

    # get valid execution plans of DAG
    valid_execution_plans = valid_execution_plans_recursive(adjacency_matrix, neighbors_dict, global_active_dict, memory_dict, [0])

    # sort number within subsets
    valid_execution_plans = [[sorted(inner_list) for inner_list in outer_list] for outer_list in valid_execution_plans]

    return valid_execution_plans

In [14]:
example_adjacency_matrix = np.array([[0, 1, 1, 0, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 0, 0],
                                     [0, 0, 0, 0, 0]])

valid_execution_plans = get_valid_execution_plans(example_adjacency_matrix)
valid_execution_plans

[[[0], [1], [3], [2]],
 [[0, 1], [3], [2]],
 [[0], [1, 3], [2]],
 [[0, 1, 3], [2]],
 [[0], [1], [2, 3]],
 [[0, 1], [2, 3]],
 [[0], [1, 2, 3]],
 [[0, 1, 2, 3]],
 [[0], [2], [3], [1]],
 [[0, 2], [3], [1]],
 [[0], [2, 3], [1]],
 [[0, 2, 3], [1]],
 [[0], [2], [1, 3]],
 [[0, 2], [1, 3]],
 [[0], [1, 2, 3]],
 [[0, 1, 2, 3]],
 [[0, 1, 2], [3]],
 [[0, 1, 2, 3]]]

In [15]:
# example_adjacency_matrix = np.array([[0, 1, 1, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 0],
#                                      [0, 0, 0, 0, 0]])

# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

In [16]:
# example_adjacency_matrix = np.array([[0, 0, 1, 0, 0],
#                                      [0, 0, 1, 0, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 1],
#                                      [0, 0, 0, 0, 0]])


# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

In [17]:
valid_execution_plans = get_valid_execution_plans(adjacency_matrix)
#valid_execution_plans

In [18]:
def canonicalize_list_of_lists(list_of_lists):
    # Sort each inner list, then sort the outer list of sorted inner lists
    return sorted([sorted(inner_list) for inner_list in list_of_lists])

def get_unique_lists(input_lists):
    unique_set = set()
    unique_lists = []
    
    for lst in input_lists:
        # Canonicalize each list by sorting both inner and outer lists
        canonical_lst = tuple(tuple(sorted(inner)) for inner in sorted(lst))
        
        # Add the canonical form to the set if it's not already present
        if canonical_lst not in unique_set:
            unique_set.add(canonical_lst)
            unique_lists.append(lst)  # Add the original list to the result
    
    return unique_lists

print(len(valid_execution_plans))
unique_combinations = get_unique_lists(valid_execution_plans)
print(len(unique_combinations))

153919
2106


### Generate all possible partition schemes for the set of tasks (ignoring structre of the DAG)

In [19]:
from itertools import combinations

def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(1, len(set_) + 1):
        for part in combinations(set_, i):
            remaining = set(set_) - set(part)
            if not remaining:
                yield [list(part)]
            else:
                for b in partitions(list(remaining)):
                    yield [list(part)] + b

def generate_unique_partitions(numbers):
    all_partitions = set()
    for partition in partitions(numbers):
        # Create a frozenset of frozensets to make each partition hashable and order-independent
        partition_set = frozenset(frozenset(part) for part in partition)
        all_partitions.add(partition_set)
    
    # Convert the frozensets back to lists for the final output
    unique_partitions = [list(map(list, partition)) for partition in all_partitions]

    # Sort elements
    unique_partitions = sorted([sorted(x) for x in unique_partitions], key=len)
    return unique_partitions

In [20]:
# Generate list of numbers for non-"Target" tasks in occupation
tasks_list_numbers = list(range(len(tasks_dict)-1))

# Generate all possible partitioning schemes
all_partitions = generate_unique_partitions(tasks_list_numbers)

### Check if partition scheme is "valid" (i.e., if its non-singleton partitions are a connected graph)

In [21]:
def is_connected(matrix):
    # Number of nodes in the matrix
    num_nodes = matrix.shape[0]
    
    # Visited array to keep track of visited nodes
    visited = np.zeros(num_nodes, dtype=bool)
    
    # Helper function to perform DFS
    def dfs(node):
        visited[node] = True
        # Visit all the neighbors of the current node
        for neighbor in range(num_nodes):
            if matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor)
            elif matrix[neighbor, node] == 1 and not visited[neighbor]:
                dfs(neighbor)
    
    # Start DFS from the first node (node 0)
    dfs(0)
    
    # If all nodes are visited, the matrix is connected
    return np.all(visited)


def validate_partition_using_connectedness(adjacency_matrix, tasks_list):
    # Return valid if Singleton
    if len(tasks_list) == 1:
        return True
    # Check if partition forms connected graph
    else:
        # Subset original adjacency matrix
        subset_matrix = adjacency_matrix[np.ix_(tasks_list, tasks_list)]

        # check if subset matrix is a connected graph
        subset_matrix_connected = is_connected(subset_matrix)

        # return true if connected and false otherwise
        return subset_matrix_connected

In [22]:
# Get valid partitioning schemes from all possible partitions to cut computation load
valid_partitions = []
for scheme in all_partitions:
    # Set valid partitions count to 0
    valid_partition_count = 0
    for partition in scheme:
        valid_partition = validate_partition_using_connectedness(adjacency_matrix, partition)
        if valid_partition:
            valid_partition_count += 1
    
    # If number of valid partitions within a partition scheme is equal to 
    # number of partitions in partition scheme then partition scheme is valid
    if valid_partition_count == len(scheme):
        valid_partitions.append(scheme)

# Print stats
print(f'Number of all possible partitioning schemes: {len(all_partitions)}')
print(f'Number of valid partitioning schemes given DAG structure: {len(valid_partitions)}')

valid_partitions

Number of all possible partitioning schemes: 4140
Number of valid partitioning schemes given DAG structure: 2106


[[[0, 1, 2, 3, 4, 5, 6, 7]],
 [[0, 1, 2, 3, 6, 7], [4, 5]],
 [[0, 2, 3, 4, 5], [1, 6, 7]],
 [[0, 2, 3, 4, 5, 6], [1, 7]],
 [[0, 4, 5, 7], [1, 2, 3, 6]],
 [[0, 3, 4, 6], [1, 2, 5, 7]],
 [[0, 1, 2, 3, 5, 6], [4, 7]],
 [[0, 1, 2, 6], [3, 4, 5, 7]],
 [[0, 1, 2, 4, 5, 6], [3, 7]],
 [[0, 1, 2, 4, 5, 6, 7], [3]],
 [[0], [1, 2, 3, 4, 5, 6, 7]],
 [[0, 2, 4, 5, 6, 7], [1, 3]],
 [[0, 1, 2, 3, 4, 6, 7], [5]],
 [[0, 2, 4], [1, 3, 5, 6, 7]],
 [[0, 2, 3, 7], [1, 4, 5, 6]],
 [[0, 1, 5, 6], [2, 3, 4, 7]],
 [[0, 2, 4, 5, 7], [1, 3, 6]],
 [[0, 1, 5, 6, 7], [2, 3, 4]],
 [[0, 1, 3, 5, 6], [2, 4, 7]],
 [[0, 1, 2, 3, 4, 5, 6], [7]],
 [[0, 2, 3, 4, 7], [1, 5, 6]],
 [[0, 1, 2, 3, 7], [4, 5, 6]],
 [[0, 1, 2, 3, 5], [4, 6, 7]],
 [[0, 3, 6, 7], [1, 2, 4, 5]],
 [[0, 1, 2, 3, 5, 7], [4, 6]],
 [[0, 1, 4, 5, 6], [2, 3, 7]],
 [[0, 3, 6], [1, 2, 4, 5, 7]],
 [[0, 1, 3, 4, 5, 6, 7], [2]],
 [[0, 1, 3, 4, 5, 7], [2, 6]],
 [[0, 1, 2, 4, 5, 7], [3, 6]],
 [[0, 6], [1, 2, 3, 4, 5, 7]],
 [[0, 1, 4, 5], [2, 3, 6, 7]],
 [[0, 2, 3

## Missing in "Smart" method

In [23]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def list_difference(list1, list2):
    # Normalize both lists
    normalized_list1 = {normalize_sublist(sublist) for sublist in list1}
    normalized_list2 = {normalize_sublist(sublist) for sublist in list2}
    
    # Find the difference
    difference = normalized_list1 - normalized_list2
    
    # Convert the normalized tuples back to the original list format
    difference_list = []
    for norm_sublist in difference:
        original_sublist = [list(inner) for inner in norm_sublist]
        difference_list.append(original_sublist)
    
    return difference_list

In [24]:
inBruteForce_notInSmart = list_difference(valid_partitions, unique_combinations)
print(f'In Brute Force but not in Smart: {len(inBruteForce_notInSmart)}\n')
for case in inBruteForce_notInSmart:
    print(case)

In Brute Force but not in Smart: 0



In [25]:
inSmart_notInBruteForce = list_difference(unique_combinations, valid_partitions)
print(f'In Smart but not in Brute Force: {len(inSmart_notInBruteForce)}\n')
for case in inSmart_notInBruteForce:
    print(case)

In Smart but not in Brute Force: 0

